# Dynamic Bayesian Network

We have now constructed the P(AU) and P(FT) as outlined in the theoretical model. Now we move on to computing the P(AU and FT) for each AU and each FT.

In [6]:
import os
import pandas as pd
import pprint 
import pickle
import numpy as np
pp = pprint.PrettyPrinter(indent=4)
from sklearn.model_selection import train_test_split
from pgmpy.models import BayesianNetwork, DynamicBayesianNetwork
from pgmpy.inference import DBNInference
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.base import DAG

In [2]:
# load data

with open("markov_probs.pickle", 'rb') as f:
    probabilities = pickle.load(f)
    
with open("onesec_dataframes.pickle", 'rb') as f:
    dataframes = pickle.load(f)
    
with open("conditional_probs.pickle", 'rb') as f:
    conditional_probabilities = pickle.load(f)
    
    

In [3]:
print(type(dataframes['/home/roni/coding/mastersProject/src/csvOut/p_100/recording_3'].values))

<class 'numpy.ndarray'>


In [13]:
G = DAG()
nodes = [' AU17_c', ' AU07_c',' AU14_c' , ' AU12_c',' AU20_c' , 'leftHandTouching', 'rightHandTouching' ]
cooc = [ 'LH AU17_c', 'LH AU07_c','LH AU14_c' , 'LH AU12_c','LH AU20_c' , 'RH AU17_c', 'RH AU07_c','RH AU14_c' , 'RH AU12_c','RH AU20_c' ]
res = ['GAD Score','PHQ Score']

G.add_nodes_from(nodes)
G.add_nodes_from(cooc)

for i in cooc:
    for j in nodes:
        if ("LH"+j == i):
            G.add_edge( u=j, v=i)
        elif ("RH"+j == i):
            G.add_edge( u=j, v=i)
        if(j == 'leftHandTouching' and i[0:2]=='LH'):
            G.add_edge( u=j, v=i)
        elif(j == 'rightHandTouching' and i[0:2]=='RH'):
            G.add_edge( u=j, v=i)
    G.add_edge( u=i, v=res[0])
    G.add_edge( u=i, v=res[1])

G.edges()

OutEdgeView([(' AU17_c', 'LH AU17_c'), (' AU17_c', 'RH AU17_c'), (' AU07_c', 'LH AU07_c'), (' AU07_c', 'RH AU07_c'), (' AU14_c', 'LH AU14_c'), (' AU14_c', 'RH AU14_c'), (' AU12_c', 'LH AU12_c'), (' AU12_c', 'RH AU12_c'), (' AU20_c', 'LH AU20_c'), (' AU20_c', 'RH AU20_c'), ('leftHandTouching', 'LH AU17_c'), ('leftHandTouching', 'LH AU07_c'), ('leftHandTouching', 'LH AU14_c'), ('leftHandTouching', 'LH AU12_c'), ('leftHandTouching', 'LH AU20_c'), ('rightHandTouching', 'RH AU17_c'), ('rightHandTouching', 'RH AU07_c'), ('rightHandTouching', 'RH AU14_c'), ('rightHandTouching', 'RH AU12_c'), ('rightHandTouching', 'RH AU20_c'), ('LH AU17_c', 'GAD Score'), ('LH AU17_c', 'PHQ Score'), ('LH AU07_c', 'GAD Score'), ('LH AU07_c', 'PHQ Score'), ('LH AU14_c', 'GAD Score'), ('LH AU14_c', 'PHQ Score'), ('LH AU12_c', 'GAD Score'), ('LH AU12_c', 'PHQ Score'), ('LH AU20_c', 'GAD Score'), ('LH AU20_c', 'PHQ Score'), ('RH AU17_c', 'GAD Score'), ('RH AU17_c', 'PHQ Score'), ('RH AU07_c', 'GAD Score'), ('RH AU0

In [12]:
model = DynamicBayesianNetwork()
nodes = [' AU17_c', ' AU07_c',' AU14_c' , ' AU12_c',' AU20_c' , 'leftHandTouching', 'rightHandTouching' ]
n = [ 'LH AU17_c', 'LH AU07_c','LH AU14_c' , 'LH AU12_c','LH AU20_c' , 'RH AU17_c', 'RH AU07_c','RH AU14_c' , 'RH AU12_c','RH AU20_c' ]

for video_name, df in dataframes.items():
    temp = BayesianNetwork(G)    
    est = MaximumLikelihoodEstimator(temp,df)
    temp.fit(df,est)
    
    #infer = DBNInference(temp)
    #estimator = MaximumLikelihoodEstimator(model, temp)
    
    #temp.fit(probabilities[video_name], estimator)#, inference_algorithm=infer)
    
    #for node_name in nodes:
    #    model.add_node(node_name, temp.cpd(node_name))
    
    #if model.cardinality is None:
    #    model.cardinality = temp.cardinality
    #else:
    #    model.cardinality = np.maximum(model.cardinality, temp.cardinality)
    
    #model.add_transition(model.previous_timestep_nodes, nodes, transition_type='sequential')
    #model.previous_timestep_nodes = nodes
    
    
    
    #model.add_node(df, temp)
    
    #add temp to model



ValueError: Maximum Likelihood Estimator works only for models with all observed variables. Found latent variables: set().

In [ ]:
# Set the first nodes as start nodes
model.add_transition(model.start_nodes, model.previous_timestep_nodes, transition_type='sequential')
    
# Add end nodes and set their transitions
end_nodes = []
for node_name in nodes:
    end_node_name = f"{node_name}_end"
    end_nodes.append(end_node_name)
    model.add_node(end_node_name)
    model.add_transition([node_name], end_node_name, transition_type='static')
model.add_transition(model.previous_timestep_nodes, end_nodes, transition_type='sequential')

model.bake()